In [1]:

from abc import ABC, abstractmethod
from typing import Any, Dict

class Tool(ABC):
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    @abstractmethod
    def __call__(self, input: Any) -> str:
        pass

    def get_info(self) -> Dict[str, str]:
        return {
            "name": self.name,
            "description": self.description
        }

class InternetSearch(Tool):
    def __init__(self):
        super().__init__(
            name="internet_search",
            description="Searches the internet for information. Input should be a search query."
        )

    def __call__(self, query: str) -> str:
        # In a real implementation, this would use an actual search API
        return f"Search results for '{query}': [Simulated search results]"

class PlayMusic(Tool):
    def __init__(self):
        super().__init__(
            name="play_music",
            description="Plays music on Spotify. Input should be a song name or artist."
        )

    def __call__(self, song_name: str) -> str:
        # In a real implementation, this would interact with the Spotify API
        return f"Playing '{song_name}' on Spotify"

class GetWeather(Tool):
    def __init__(self):
        super().__init__(
            name="get_weather",
            description="Retrieves weather information for a specific city. Input should be a city name."
        )

    def __call__(self, city_name: str) -> str:
        # In a real implementation, this would use a weather API
        return f"Weather in {city_name}: Sunny, 22°C"

def get_all_tool_info(tools: list[Tool]) -> list[Dict[str, str]]:
    return [tool.get_info() for tool in tools]

In [2]:
tools = [InternetSearch(), PlayMusic(), GetWeather()]

for tool_info in get_all_tool_info(tools):
    print(f"- {tool_info['name']}: {tool_info['description']}")

- internet_search: Searches the internet for information. Input should be a search query.
- play_music: Plays music on Spotify. Input should be a song name or artist.
- get_weather: Retrieves weather information for a specific city. Input should be a city name.


In [11]:
import time
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os

load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

# Set up authentication
scope = "user-library-read user-read-playback-state user-modify-playback-state"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET, redirect_uri="http://localhost:8888/callback"))

def get_active_device():
    devices = sp.devices()
    print("Available devices:")
    for i, device in enumerate(devices['devices']):
        print(f"{i+1}. {device['name']} ({'Active' if device['is_active'] else 'Inactive'})")
    
    if not devices['devices']:
        return None
    
    choice = input("Enter the number of the device you want to use (or press Enter for the first one): ")
    if choice.isdigit() and 1 <= int(choice) <= len(devices['devices']):
        return devices['devices'][int(choice)-1]['id']
    return devices['devices'][0]['id']

import subprocess

def search_and_play(query, type, retry=False):
    print(f"Searching for {type}: {query}")
    # Search for the item
    results = sp.search(q=query, type=type)
    
    if len(results[f"{type}s"]['items']) == 0:
        print(f"No {type} found for the query: {query}")
        return
    
    # Get the first result
    item = results[f"{type}s"]['items'][0]
    item_name = item['name']
    item_uri = item['uri']
    
    print(f"Found {type}: {item_name}")
    
    # Get available devices
    devices = sp.devices()
    if not devices['devices']:
        if not retry:
            print("No active devices found. Attempting to open Spotify...")
            subprocess.Popen(['spotify'], start_new_session=True)
            print("Waiting for Spotify to start...")
            time.sleep(4)  # Wait to allow Spotify to start
            return search_and_play(query, type, retry=True)
        else:
            print("Failed to find an active device after opening Spotify.")
            print("Please ensure Spotify is running and a device is active.")
            return
    
    # Use the first available device
    device_id = devices['devices'][0]['id']
    
    # Start playback
    try:
        print(f"Attempting to play on device: {device_id}")
        if type == 'track':
            sp.start_playback(device_id=device_id, uris=[item_uri])
        else:
            sp.start_playback(device_id=device_id, context_uri=item_uri)
        print(f"Now playing: {item_name}")
    except spotipy.exceptions.SpotifyException as e:
        print(f"Error starting playback: {e}")
        print("This error may occur if Spotify is not active on the selected device.")
        print("Try manually starting playback on your device, then run this script again.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print("Please check your internet connection and Spotify account status.")

In [13]:
search_and_play("On me lil baby", "track")
#search_and_play("Vendido", "track")

Searching for track: On me lil baby
Found track: On Me
No active devices found. Attempting to open Spotify...
Waiting for Spotify to start...


Gtk-Message: 16:50:19.107: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


Searching for track: On me lil baby
Found track: On Me
Attempting to play on device: 773551dda2ddd213eb91b4264709a7dca199e68b
Now playing: On Me
